In [1]:
import torchvision
from model import *

train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练集大小：{}".format(train_data_size))

Files already downloaded and verified
Files already downloaded and verified
训练集大小：50000


In [2]:
from torch.utils.data import DataLoader
#加载数据
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [7]:
#创建网络模型
tudui = TuDui()
if torch.cuda.is_available():
    tudui = tudui.cuda()

#损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()

#优化器
#1e-2=10^-2=0.01
learning_rate = 1e-2
optimizer = torch.optim.Adam(tudui.parameters(), lr=learning_rate)

#设置训练网络的参数
#记录训练的次数
total_train_step = 0
#记录测试的次数
total_test_step = 0
#训练的轮数
epoch = 10

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("logss")

import time
start_time = time.time()

for i in range(epoch):
    print(f"Epoch {i+1}")
    
    #训练步骤开始
    for data in train_dataloader:
        imgs, targets = data 
        imgs = imgs.cuda()
        targets = targets.cuda()
        outputs = tudui(imgs)
        loss = loss_fn(outputs, targets)
        
        #优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print(f"Train Step {total_train_step}, Loss: {loss.item():.4f}")
            writer.add_scalar("Train Loss", loss.item(), total_train_step)
       
    #测试步骤开始
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data 
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs = tudui(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(dim=1) == targets).sum()
            total_accuracy += accuracy.item()
    print(f"Test Loss: {total_test_loss:.4f}, Accuracy: {total_accuracy/test_data_size:.4f}")
    writer.add_scalar("Test Loss", total_test_loss, total_test_step)
    writer.add_scalar("Test Accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1
    
    torch.save(tudui, "tudui_{}.pth".format(i))
    # torch.save(tudui.state_dict(), "tudui_{}.pth".format(i))
    print("Model Saved")
writer.close()

Epoch 1
1.239436149597168
Train Step 100, Loss: 2.0322
2.1964526176452637
Train Step 200, Loss: 1.9818
3.14658522605896
Train Step 300, Loss: 1.7230
4.079493522644043
Train Step 400, Loss: 1.4943
5.008159160614014
Train Step 500, Loss: 1.5174
5.938474893569946
Train Step 600, Loss: 1.6225
6.874080181121826
Train Step 700, Loss: 1.5270
Test Loss: 228.1578, Accuracy: 0.4813
Model Saved
Epoch 2
9.127981901168823
Train Step 800, Loss: 1.3143
10.065816402435303
Train Step 900, Loss: 1.2914
10.997817277908325
Train Step 1000, Loss: 1.4033
11.914487361907959
Train Step 1100, Loss: 1.5322
12.840702295303345
Train Step 1200, Loss: 1.3463
13.764557123184204
Train Step 1300, Loss: 1.0978
14.694077968597412
Train Step 1400, Loss: 0.9965
15.607791185379028
Train Step 1500, Loss: 1.1757
Test Loss: 185.5052, Accuracy: 0.5890
Model Saved
Epoch 3
17.864424467086792
Train Step 1600, Loss: 1.0794
18.79596209526062
Train Step 1700, Loss: 1.1086
19.743156671524048
Train Step 1800, Loss: 1.3078
20.674239873

https://colab.research.google.com  免费使用GPU

In [ ]:
#方法一
.cuda()

#方法二
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)